In [ ]:
#importa as bibliotecas
import pandas as pd
import yfinance as yf
from datetime import datetime


"""import pandas_datareader.data as pdr: (antes era utilizado, hoje não é mais)
yf.pdr_override(): (O código original utiliza a função abaixo, mas ela foi removida do módulo yfinance. Então hoje para acessarmos dados 
do Yahoo Finance é necessário apenas do yfinance, sem necessitar do datareader pra 'forçar' a utilização da API do yfinance)."""


In [ ]:
"""Esse bloco importa os ativos por meio do arquivo txt e cria um dicionário correspondente ao nome e valor de cada ativo."""

#abre o arquivo "carteira.txt" na função de leitura, e o define como uma variável.
with open("carteira.txt", "r") as arquivo:
    # define a variável 'texto' e utiliza a função readlines, que cria uma lista separa cada linha como um item único
    texto = arquivo.readlines()

#criando dicionário vazio
carteira = {}

#fazendo cada linha do arquivo aderir às condições que estão no for até as linhas acabarem
for linha in texto:
    #a função 'split' separa a linha em duas variáveis, usando o caractere '-' como divisão;
    ticker, valor = linha.split("-")

    #faz com que a variável 'ticker' tenha espaços removidos e colocando ".SA" no fim de cada ticket.
    ticker = f"{ticker.strip()}.SA"

    #aplicou o strip e transformou a variável em valor numérico.
    valor = float(valor.strip())

    #adiciona as variáveis ao dicionário, atribuindo chave como 'ticker' e 'valor' como o value correspondente a chave.
    carteira[ticker] = valor
print(carteira)

#Tickets de ações tem ".SA" no final - ITUB4.SA
#Índices tem "^" no começo - ^BVSP

In [ ]:
"""Este bloco obtém do Yahoo Finance os dados de rentabilidade das ações presentes no dicionário para um período de tempo específico e adiciona o índice Bovespa ao dicionário para comparações futuras."""

#cria uma lista com todas as chaves do dicionário
ativos = list(carteira.keys())

#adiciona o índice Bovespa na lista para comparação
ativos.append("^BVSP")

#estabelece uma variável para data inicial
datainicial = "2024-01-01"

#estabelece a data final, setando o valor como a data/hora atual e definindo formato como ano, mês, dia
data_final = datetime.now().strftime('%Y-%m-%d')

#importa as cotações da lista de ações 'ativos'
tabela_cotacoes= yf.download(ativos, start=datainicial, end=data_final, auto_adjust=False)

#define que só a coluna "Adj Close" aparecerá
tabela_cotacoes = tabela_cotacoes["Adj Close"]

#mostra tabela
display(tabela_cotacoes)


In [ ]:
"""Este bloco calcula a rentabilidade de cada ativo com base nos valores inicial e final e os armazena em um dicionário."""

#cria dicionário vazio
rentabilidades = {}

#cada coluna da 'tabela_cotacoes' irá aderir às condições abaixo
for ativo in tabela_cotacoes.columns:

    #define a variável "rentabilidade" de cada ativo, que é igual à ultima linha dividida pela primeira 
    rentabilidade = tabela_cotacoes[ativo][-1] / tabela_cotacoes[ativo][0]

    #transforma a 'rentabilidade' em float e a adiciona no dicionário para cada ativo
    rentabilidades[ativo] = float(rentabilidade)

#exibe o dicionário de rentabilidades correspondente a cada ativo
print (rentabilidades)

In [ ]:
"""Este bloco de código compara a rentabilidade da carteira de investimentos em relação ao índice Bovespa (^BVSP) no período de tempo que foi definido."""

#soma os valores correspondentes a todas as chaves do dicionário "carteira" e o atribui à variável "valor_inicial"
valor_inicial = sum(carteira.values())

#exibe uma f-string com o valor inicial da carteira formatado para exibir duas casas decimais
print (f"O valor inicial da sua carteira era: R${valor_inicial:,.2f}")

#valor final = soma de todos os ativos da carteira aplicando a fórmula (valor inicial de cada ativo * rentabilidade)
valor_final = sum (carteira[ativo] * rentabilidades [ativo] for ativo in carteira)

#exibe uma f-string com o valor final da carteira formatado para exibir duas casas decimais
print (f"Após o período, o valor final da sua carteira ficou em: R${valor_final:,.2f}")

#rentabilidade da carteira corresponde ao valor inicial dividido pelo valor inicial -1 
rentabilidade_carteira = valor_final / valor_inicial -1

#exibe o valor da rentabilidade da carteira em porcentagem definindo duas casas decimais
print (f"Durante esse período, a rentabilidade total da sua carteira foi: {rentabilidade_carteira:.2%}")

# rentabilidade do índice corresponde ao valor da coluna "^BVSP" - 1 no dicionário de rentabilidades
rentabilidade_indice = rentabilidades["^BVSP"] -1

#exibe o valor da rentabilidade do índice em porcentagem definindo duas casas decimais
print(f"Durante esse período, a rentabilidade total da sua carteira foi: {rentabilidade_indice:.2%}")

#se a rentabilidade da carteira for maior que a do índice, exibir mensagem
if rentabilidade_carteira > rentabilidade_indice:
    print ("Portanto, a rentabilidade da sua carteira foi maior que a do índice Bovespa.")

#se a rentabilidade da carteira for igual que a do índice, exibir mensagem
elif rentabilidade_carteira == rentabilidade_indice:
    print ("Portanto, a rentabilidade da sua carteira foi igual a do índice Bovespa.")

#se a rentabilidade da carteira for menor que a do índice, exibir mensagem
else:
    print ("Portanto, a rentabilidade da sua carteira foi menor que a do índice Bovespa.")